In [37]:
import rasterio
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Point, mapping
import json

In [4]:
clayData = rasterio.open('./clay.tif')
sandData = rasterio.open('./sand.tif')
densityData = rasterio.open('./density.tif')
moistureData = rasterio.open('./soil_moisture.tif')

In [5]:
fields = gpd.read_file('field_centroids.geojson')

In [42]:
field_names = []
field_centroids = []
clay_content = []
sand_content = []
soil_density = []
soil_moisture = []

for index, field in fields.iterrows():
    field_name = field['Name']
    field_names.append(field_name)
    centroid = field.geometry.centroid
    field_centroids.append(centroid)
    
    row, col = moistureData.index(centroid.x, centroid.y)
    moisture_val = moistureData.read(1)[row, col]
    soil_moisture.append(moisture_val)
    
    row, col = clayData.index(centroid.x, centroid.y)
    clay_val = clayData.read(1)[row, col]
    clay_content.append(clay_val)
    
    row, col = sandData.index(centroid.x, centroid.y)
    sand_val = sandData.read(1)[row, col]
    sand_content.append(sand_val)
    
    row, col = densityData.index(centroid.x, centroid.y)
    density_val = densityData.read(1)[row, col]
    soil_density.append(density_val)

In [45]:
#Тут я спочатку зробив, а потім глянува як воно має виглядати
data = {
    'Field_Name': field_names,
    'Field_Center': field_centroids,
    'Clay_Content': clay_content,
    'Sand_Content': sand_content,
    'Soil_Density': soil_density,
    'Soil_Moisture': soil_moisture
}
df = pd.DataFrame(data)
df.head(3)


,Field_Name,Field_Center,Clay_Content,Sand_Content,Soil_Density,Soil_Moisture
0,Pin4,POINT (30.184082372130998 49.995266264307425),21,38,1149,2785
1,Pin17,POINT (30.250268297428843 49.98931843919832),21,37,1212,2813
2,Shev7-8-9,POINT (30.333908293755627 50.01055996618192),22,35,1184,3158


In [12]:
field_centroids = gpd.read_file('field_centroids.geojson')
output_data = []


In [39]:
#Тут в мене не коректно виводить дані :(

for index, row in field_centroids.iterrows():

    lat = row.geometry.y
    lon = row.geometry.x
    clay = next(rasterio.open('./clay.tif').sample([(lon, lat)]))
    sand = next(rasterio.open('./sand.tif').sample([(lon, lat)]))
    density = next(rasterio.open('./density.tif').sample([(lon, lat)]))
    soil_moisture = next(rasterio.open('./soil_moisture.tif').sample([(lon, lat)]))
    
    
    field_data = {
        "type": "Feature",
        "properties": {
            "id": index + 1,
            "name": field_name,
            "soil": {
                "clay": clay,
                "sand": sand,
                "density": density
            },
            "soil_moisture": soil_moisture
        },
        "geometry": {
            "type": "Point",
            "coordinates": [lon, lat]
        }
    }  
    
    
    output_data.append(field_data)
output_data.pop(3)



{'type': 'Feature',
 'properties': {'id': 44,
  'name': 'Ks13',
  'soil': {'clay': <generator object sample_gen at 0x00000278A39F6EB0>,
   'sand': <generator object sample_gen at 0x00000278A3A0B430>,
   'density': <generator object sample_gen at 0x00000278A39F6F20>},
  'soil_moisture': <generator object sample_gen at 0x00000278A3A0B740>,
  'coordinates': {'lat': 50.03570104293429, 'lng': 30.220735144875057}},
 'geometry': {'type': 'Point',
  'coordinates': [30.220735144875057, 50.03570104293429]}}

https://github.com/BogdanJeN/Geo/blob/main/Lab_06/lab_06.ipynb

In [48]:
clayData = rasterio.open('./clay.tif')
sandData = rasterio.open('./sand.tif')
densityData = rasterio.open('./density.tif')
moistureData = rasterio.open('./soil_moisture.tif')
fieldsData = gpd.read_file("./field_centroids.geojson")

In [50]:
field_list = []

for index, row in fieldsData.iterrows():
    id = row['id']
    name = row['Name']
    centroid = row['geometry'].centroid

    lon, lat = centroid.x, centroid.y

    row, col = moistureData.index(lon, lat)
    clay = clayData.read(1)[row, col]
    sand = sandData.read(1)[row, col]
    density = densityData.read(1)[row, col]

    moisture = moistureData.read(1)[row, col]

    features = {
        'type': 'Feature',
        'properties': {
            'id': id,
            'name': name,
            'soil': {
                'clay': float(clay),
                'sand': float(sand),
                'density': int(density)
            },
            'soil_moisture': int(moisture),
            'coordinates': {
                'lat': float(lat),
                'lng': float(lon)
            }
        },
        'geometry': mapping(centroid)
    }

    field_list.append(features)

feature_collection = {
    'type': 'FeatureCollection',
    'features': field_list
}

with open('./field_data.geojson', 'w') as f:
    json.dump(feature_collection, f, indent=4)